In [1]:
class vector:
    def __init__(self,color,weight,start,end):
        self.color=color
        self.weight=weight
        self.start=start
        self.end=end
    def work(self):
        if self.start.activated:  
            if self.color=="red":
                if self.start.totake>0:
                    self.end.value+=self.start.totake
                    self.end.nextactivated=1
                
            elif self.color=="green":
                self.end.value+=self.start.totake*self.weight
                self.end.nextactivated=1

            else:
                self.end.value+=self.weight
                self.end.nextactivated=1
        

In [2]:
class node:
    def __init__(self,name):
        self.activated=0
        self.nextactivated=0
        self.value=0
        self.totake=0
        self.name=name
    def work(self):
        self.activated=self.nextactivated
        self.nextactivated=0
        self.value-=self.totake
        self.totake=self.value
       

In [3]:
neurons=[]
synapses=[]
for i in range(3):
    synapses.append(node(i))
neurons.append(vector("red",0,synapses[0],synapses[1]))
neurons.append(vector("blue",2,synapses[0],synapses[2]))
neurons.append(vector("green",3,synapses[2],synapses[1]))

In [4]:
def run(brain,FIRING,show="no"):
    tot=0
    for i in range(FIRING):
        if show=="yes":
            for synapse in brain[0]:
                print(synapse.value)
            print("\n")
        for neuron in brain[1]:
            neuron.work()
        for synapse in brain[0]:
            synapse.work()

        tot+=brain[0][0].value
        
    return tot

In [5]:
import copy
import random
colors=["red","green","blue"]
def addvector(brain):
    torepeat=True
    c=0
    while(torepeat and c<100):
        torepeat=False
        start=0
        end=0
        c+=1
        while(start==end):
            start=random.randint(0,len(brain[0])-1)
            end=random.randint(0,len(brain[0])-1)
        color=random.randint(0,2)
        for neuron in brain[1]:
            if neuron.color==colors[color] and neuron.start==brain[0][start] and neuron.end==brain[0][end]:
                torepeat=True
                break
         
    brain[1].append(vector(colors[color],random.randint(-4,4),brain[0][start],brain[0][end]))
    #se non trova uno spazio libero looppa fino a 100 implementazione mediocre
def remvector(brain):
    start=random.randint(0,len(brain[1])-1)
    brain[1].pop(start)
def changeweight(brain,change=2):
    neurontochange=random.randint(0,len(brain[1])-1)
    brain[1][neurontochange].weight+=round(random.uniform(-1,1),2)*change
    
def info(brain):
    for neuron in brain[1]:
        print(f"\n{neuron.color} {neuron.start.name} {neuron.end.name} ", end = '')
        if neuron.color!="red":
            print(neuron.weight, end = '')
def changestart(brain):
    start=random.randint(0,len(brain[1])-1)
    brain[1][start].start=brain[0][random.randint(0,len(brain[0])-1)]
def changend(brain):
    start=random.randint(0,len(brain[1])-1)
    brain[1][start].end=brain[0][random.randint(0,len(brain[0])-1)]
def changecolor(brain):
    start=random.randint(0,len(brain[1])-1)
    brain[1][start].color=colors[random.randint(0,2)]
def addnode(brain):
    brain[0].append(node(len(brain[0])))
def resetbrain(brains):
    for j in range(len(brains[0])):
        brains[0][j].value=0
        brains[0][j].totake=0
        brains[0][j].activated=0
        brains[0][j].nextactivation=0

In [ ]:
brains=[]
for i in range(8):
    brains.append([])
    brains[i].append([])
    for a in range(5):
        brains[i][0].append(node(a))
    brains[i].append([])
    
    for a in range(8):
        addvector(brains[i])

    #info(brains[i])
    print("\n")
generation=0
continuare="1"
while(continuare=="1"):
    print(f"generazione: {generation}\n")
    scores=[]
    
    #fitness evaluation
    for i in range(len(brains)):
        info(brains[i])
        print("\n\n")
        brains[i][0][1].value=4
        brains[i][0][1].totake=4
        brains[i][0][1].activated=1
        scores.append(run(brains[i],12))
        resetbrain(brains[i])   
    for i in range(len(brains)):
        print(scores[i])
    #fitness evaluation 
    
    #next generation
    tosave=0
    newbrain=[]
    while(tosave>=0):
        tosave=int(input("scelta: "))
        if(tosave<len(brains)and tosave>=0):
            g=copy.deepcopy(brains[tosave])
            newbrain.append(g)
        elif(tosave>=len(brains)):
            print("scelta troppo grande")
    tosave=int(input("scelta 0> <0: "))
    while(tosave>=0):
        notvalid=1
        while(notvalid):
            madre=int(input("madre: "))
            padre=int(input("padre: "))
            if(madre>=len(brains) or padre>=len(brains)):
                print("out of range")
            else:
                notvalid=0
        g=max(len(brains[madre][1]),len(brains[padre][1]))
        s=min(len(brains[madre][1]),len(brains[padre][1]))
        cut=random.randint(1,s-1)
        for f in range(g-cut):
            if cut+f<len(brains[madre][1]):
                buffer=copy.deepcopy(brains[madre][1][cut])

                if(buffer.start.name>=len(brains[padre][1])):
                    brains[padre][0].append(node(len(brains[padre][1])))
                    buffer.start=brains[padre][0][-1]
                if(buffer.end.name>=len(brains[padre][1])):
                    brains[padre][0].append(node(len(brains[padre][1])))
                    buffer.end=brains[padre][0][-1]

                brains[padre][1].append(buffer)
                brains[padre][1][-1].start=brains[padre][0][brains[padre][1][-1].start.name]
                brains[padre][1][-1].end=brains[padre][0][brains[padre][1][-1].end.name]
            if cut+f<len(brains[padre][1]):
                bufferpadre=copy.deepcopy(brains[padre][1][cut])
                if(buffer.start.name>=len(brains[madre][1])):
                    brains[madre][0].append(node(len(brains[madre][1])))
                    buffer.start=brains[madre][0][-1]
                if(buffer.end.name>=len(brains[madre][1])):
                    brains[madre][0].append(node(len(brains[madre][1])))
                    buffer.end=brains[madre][0][-1]

                brains[madre][1].append(bufferpadre)
                brains[madre][1][-1].start=brains[madre][0][brains[madre][1][-1].start.name]
                brains[madre][1][-1].end=brains[madre][0][brains[madre][1][-1].end.name]


    #         brains[padre][1].append(buffer)
    #         brains[padre][1][-1].start=brains[padre][0][brains[padre][1][cut+f].start.name]
    #         brains[padre][1][-1].end=brains[padre][0][brains[padre][1][cut+f].end.name]
            if cut+f<len(brains[madre][1]):
                brains[madre][1].pop(cut)
            if cut+f<len(brains[padre][1]):   
                brains[padre][1].pop(cut)
        #start e end potrebbero non esserci in caso aggiungi nodi,index out of range se un genitore ha più vettori di un altro
        tosave=int(input("scelta 0> <0: "))
        #LEGGI SOTTO PER RIMEDIO START END
        #se start e end non puoi semplicemente aggiungerli perchè se end name è 8 e lo aggiungi i nomi diventano 0 1  2 3 4 8 
        #darà errori poichè usi i nomi in ordine per accedere quindi devi modificare il nome da 8 a len(brains[madre][0])
     #next generation
        ma=copy.deepcopy(brains[madre])
        pa=copy.deepcopy(brains[padre])
        newbrain.append(ma)
        newbrain.append(pa)
    #mutation
    generation+=1
    brains=copy.deepcopy(newbrain)
    for i in range(len(brains)):
        for j in range(1):
            choice=random.randint(0,6)
            if choice==0:
                addvector(brains[i])
            if choice==1:
                
                remvector(brains[i])
                #lascia almeno un neurone
            if choice==2:
                changeweight(brains[i])   
            if choice==3:
                addnode(brains[i])
            if choice==4:
                changend(brains[i])
            if choice==5:
                changestart(brains[i])
            if choice==6:
                changecolor(brains[i])
                
            #no mutazione rimuovi nodi sto bene così in ogni caso devono diventare più intelligenti
            #non ritardati è buono avere pochi nodi all'inizio poichè così l'input e output si possono trovare 
            #facilmente, dando risultati fin dall'inizio, ma dopo bisogna crescere
            
    #mutation
    continuare=input("continuare? 1/0")
#riproduzione funziona solo una volta poichè modifica la lista originale  ¯\_(ツ)_/¯ 
# quindi no poligamia

















generazione: 0


blue 3 0 1
red 0 4 
red 3 4 
green 1 3 -1
red 4 1 
blue 4 1 0
blue 3 1 -4
green 1 0 -1



green 1 2 -3
red 0 3 
green 0 1 1
green 0 3 -4
blue 3 2 3
red 4 2 
blue 4 0 4
blue 2 4 0



green 3 1 2
red 3 1 
green 4 0 3
red 3 4 
green 1 2 0
green 3 0 -4
blue 4 2 -2
blue 3 0 -4



red 3 4 
green 0 1 -1
green 2 4 2
red 2 3 
green 0 4 2
green 1 4 2
green 4 2 -3
red 0 1 



blue 1 3 -1
green 3 1 -1
green 4 2 0
green 3 4 -3
green 4 3 3
red 2 4 
red 2 3 
red 1 2 



blue 3 4 -4
green 3 2 0
red 3 2 
red 1 3 
red 1 2 
blue 4 3 3
blue 3 0 4
green 4 0 3



red 0 1 
red 3 4 
blue 0 3 -2
red 0 4 
blue 3 4 1
blue 4 3 2
green 2 0 -1
blue 3 0 3



green 4 2 -3
green 3 0 3
green 4 3 1
blue 4 2 -4
blue 4 1 3
red 4 3 
red 2 3 
blue 4 0 3


5
12
0
0
0
-36
0
0
scelta: 0
scelta: 0
scelta: 0
scelta: 0
scelta: -4
scelta 0> <0: 0
madre: 1
padre: 0
scelta 0> <0: -3
continuare? 1/01
generazione: 1


blue 3 0 1
red 0 4 
green 1 3 -1
red 4 1 
blue 4 1 0
blue 3 1 -4
green 1 0 -1



blue

In [ ]:
a=[2,3,4,5,6,7]
a.pop(3)
a

In [ ]:
metti due modalità c per coppia e s per singolo così potrai scegliere gli individui per prossima generazione 
manualmente

In [23]:
import numpy 
neurons=[]
synapses=[]
for i in range(5):
    synapses.append(node(i))
neurons.append(vector("red",0,synapses[1],synapses[2]))
neurons.append(vector("blue",-1.7,synapses[3],synapses[2]))
neurons.append(vector("red",3,synapses[3],synapses[2]))
neurons.append(vector("blue",1.16,synapses[0],synapses[2]))
neurons.append(vector("red",2,synapses[2],synapses[3]))
neurons.append(vector("green",3.14,synapses[1],synapses[4]))
neurons.append(vector("red",0,synapses[2],synapses[0]))
neurons.append(vector("blue",-1.18,synapses[4],synapses[0]))

s=4
j=[]
j.append(synapses)
j.append(neurons)
j[0][1].value=s
j[0][1].totake=s
j[0][1].activated=1
print(run(j,24,"yes"))
for i in range (5):
    j[0][i].value=0
    j[0][i].totake=0
    j[0][i].activated=0
    j[0][i].nextactivation=0
# for res in brains[i][0]:
#     res.value=0
#     res.totake=0
#     res.activated=0
#     res.nextactivation=0

0
4
0
0
0


0
0
4
0
12.56


2.8200000000000003
0
0
4
0.0


0.0
0
3.46
0
0.0


3.46
0
0.0
3.46
0.0


0.0
0
2.92
0.0
0.0


2.92
0
0.0
2.92
0.0


0.0
0
2.38
0.0
0.0


2.38
0
0.0
2.38
0.0


0.0
0
1.8399999999999999
0.0
0.0


1.8399999999999999
0
0.0
1.8399999999999999
0.0


0.0
0
1.2999999999999998
0.0
0.0


1.2999999999999998
0
0.0
1.2999999999999998
0.0


0.0
0
0.7599999999999998
0.0
0.0


0.7599999999999998
0
0.0
0.7599999999999998
0.0


0.0
0
0.21999999999999975
0.0
0.0


0.21999999999999975
0
0.0
0.21999999999999975
0.0


0.0
0
-0.3200000000000003
0.0
0.0


0.0
0
0.0
0.0
0.0


0.0
0
0.0
0.0
0.0


0.0
0
0.0
0.0
0.0


0.0
0
0.0
0.0
0.0


0.0
0
0.0
0.0
0.0


0.0
0
0.0
0.0
0.0


15.7


In [52]:
brains


[]